<a href="https://colab.research.google.com/github/kridtapon/WFO-Aroon-MACD-Breakout-Strategy/blob/main/WFO_Aroon_MACD_Breakout_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: numba
    Found existing installation: numba 0.60.0
    Uninstalling numba-0.60.0:
      Successfully uninsta

In [12]:
import itertools
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Aroon Indicator
def aroon_indicator(df, period=14):
    df['Aroon_Up'] = ((df['High'].rolling(window=period).apply(lambda x: np.argmax(x) + 1) / period) * 100)
    df['Aroon_Down'] = ((df['Low'].rolling(window=period).apply(lambda x: np.argmin(x) + 1) / period) * 100)
    return df['Aroon_Up'], df['Aroon_Down']

# MACD Indicator
def macd(df, fast_period=12, slow_period=26, signal_period=9):
    df['EMA_Fast'] = df['Close'].ewm(span=fast_period, adjust=False).mean()
    df['EMA_Slow'] = df['Close'].ewm(span=slow_period, adjust=False).mean()
    df['MACD'] = df['EMA_Fast'] - df['EMA_Slow']
    df['MACD_Signal'] = df['MACD'].ewm(span=signal_period, adjust=False).mean()
    return df['MACD'], df['MACD_Signal']

# Walk-forward optimization with dynamic parameter ranges
def walk_forward_optimization(df, start_year, end_year):
    results = []

    # Define dynamic ranges for parameters
    aroon_period_range = range(10, 31, 5)
    fast_period_range = range(5, 21, 5)
    slow_period_range = range(20, 31, 2)
    macd_signal_range = range(5, 21, 5)

    for test_year in range(start_year + 4, end_year + 1):
        train_start = test_year - 4
        train_end = test_year - 1
        test_start = test_year

        train_data = df[(df.index.year >= train_start) & (df.index.year <= train_end)]
        test_data = df[df.index.year == test_year]

        best_params = None
        best_performance = -np.inf

        # Use itertools to loop through all combinations dynamically
        for params in itertools.product(
            aroon_period_range,
            fast_period_range,
            slow_period_range,
            macd_signal_range
        ):
            aroon_period, fast_period, slow_period, macd_signal = params

            # Calculate indicators on the training data
            train_data['Aroon_Up'], train_data['Aroon_Down'] = aroon_indicator(train_data.copy(), period=aroon_period)
            train_data['MACD'], train_data['MACD_Signal'] = macd(train_data.copy(), fast_period=fast_period, slow_period=slow_period, signal_period=macd_signal)

            # Generate entry and exit signals
            entries = (
                (train_data['Aroon_Down'] < train_data['Aroon_Up']) &  # Aroon Down < Aroon Up
                (train_data['MACD'] > train_data['MACD_Signal'])  # MACD Line crosses above Signal Line
            )
            exits = (
                (train_data['Aroon_Down'] > train_data['Aroon_Up']) &  # Aroon Down > Aroon Up
                (train_data['MACD'] < train_data['MACD_Signal'])  # MACD Line crosses below Signal Line
            )

            # Backtest on training data
            portfolio = vbt.Portfolio.from_signals(
                close=train_data['Close'],
                entries=entries,
                exits=exits,
                init_cash=100_000,
                fees=0.001
            )

            performance = portfolio.total_return()
            if performance > best_performance:
                best_performance = performance
                best_params = (aroon_period, fast_period, slow_period, macd_signal)

        # Test with the best parameters on the test data
        test_data['Aroon_Up'], test_data['Aroon_Down'] = aroon_indicator(test_data.copy(), period=best_params[0])
        test_data['MACD'], test_data['MACD_Signal'] = macd(test_data.copy(), fast_period=best_params[1], slow_period=best_params[2], signal_period=best_params[3])

        entries = (
            (test_data['Aroon_Down'] < test_data['Aroon_Up']) &  # Aroon Down < Aroon Up
            (test_data['MACD'] > test_data['MACD_Signal'])  # MACD Line crosses above Signal Line
        )
        exits = (
            (test_data['Aroon_Down'] > test_data['Aroon_Up']) &  # Aroon Down > Aroon Up
            (test_data['MACD'] < test_data['MACD_Signal'])  # MACD Line crosses below Signal Line
        )

        portfolio = vbt.Portfolio.from_signals(
            close=test_data['Close'],
            entries=entries,
            exits=exits,
            init_cash=100_000,
            fees=0.001
        )

        results.append({
            'Year': test_year,
            'Best_Params': best_params,
            'Test_Return': portfolio.total_return()
        })

        # Print the performance for the year
        print(f"Year {test_year}: Walk-Forward Optimization Return = {portfolio.total_return():.2f}")

    return pd.DataFrame(results)

# Define the stock symbol and time period
symbol = 'RCL'
start_date = '2015-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Perform walk-forward optimization
results = walk_forward_optimization(df, 2016, 2025)

# Display results
print("\nWalk-Forward Optimization Results:")
print(results)


Streaming output truncated to the last 5000 lines.
<ipython-input-12-cd22a37de2e5>:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-12-cd22a37de2e5>:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-12-cd22a37de2e5>:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

Year 2020: Walk-Forward Optimization Return = 0.36


Streaming output truncated to the last 5000 lines.
<ipython-input-12-cd22a37de2e5>:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-12-cd22a37de2e5>:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-12-cd22a37de2e5>:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

Year 2021: Walk-Forward Optimization Return = -0.05


Streaming output truncated to the last 5000 lines.
<ipython-input-12-cd22a37de2e5>:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-12-cd22a37de2e5>:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-12-cd22a37de2e5>:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

...

Year 2023: Walk-Forward Optimization Return = 1.45


Streaming output truncated to the last 5000 lines.
<ipython-input-12-cd22a37de2e5>:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-12-cd22a37de2e5>:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-12-cd22a37de2e5>:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

Year 2024: Walk-Forward Optimization Return = 0.56


Streaming output truncated to the last 5000 lines.
<ipython-input-12-cd22a37de2e5>:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-12-cd22a37de2e5>:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-12-cd22a37de2e5>:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

Year 2025: Walk-Forward Optimization Return = 0.00

Walk-Forward Optimization Results:
   Year       Best_Params  Test_Return
0  2020    (10, 5, 30, 5)     0.359278
1  2021  (10, 20, 26, 20)    -0.046087
2  2022  (15, 15, 24, 15)     0.258546
3  2023  (15, 15, 24, 20)     1.446193
4  2024  (15, 15, 26, 20)     0.557019
5  2025  (15, 15, 22, 15)     0.000000


In [13]:
# Combine signals into a single portfolio
combined_entries = pd.Series(False, index=df.index)
combined_exits = pd.Series(False, index=df.index)

for _, row in results.iterrows():
    year = row['Year']
    params = row['Best_Params']
    yearly_data = df[df.index.year == year]

    # Apply the Aroon indicator
    aroon_up, aroon_down = aroon_indicator(yearly_data, period=params[0])  # First param for Aroon period
    yearly_data['Aroon_Up'] = aroon_up
    yearly_data['Aroon_Down'] = aroon_down

    # Apply MACD indicator
    yearly_data['MACD'], yearly_data['MACD_Signal'] = macd(yearly_data, fast_period=params[1], slow_period=params[2], signal_period=params[3])

    # Define entry/exit conditions
    entries = (yearly_data['Aroon_Up'] > yearly_data['Aroon_Down']) & (yearly_data['MACD'] > yearly_data['MACD_Signal'])
    exits = (yearly_data['Aroon_Up'] < yearly_data['Aroon_Down']) & (yearly_data['MACD'] < yearly_data['MACD_Signal'])

    combined_entries.loc[entries.index] = entries
    combined_exits.loc[exits.index] = exits

# Filter data for testing period only
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]
combined_entries = combined_entries[(combined_entries.index.year >= 2020) & (combined_entries.index.year <= 2025)]
combined_exits = combined_exits[(combined_exits.index.year >= 2020) & (combined_exits.index.year <= 2025)]

# Backtest combined strategy
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=combined_entries,
    exits=combined_exits,
    init_cash=100_000,
    fees=0.001
)

# Display stats and plot
print(portfolio.stats())
portfolio.plot().show()

<ipython-input-12-cd22a37de2e5>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-12-cd22a37de2e5>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-13-a0e6c96a7540>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-13-a0e6c

Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           601271.666359
Total Return [%]                       501.271666
Benchmark Return [%]                    71.325669
Max Gross Exposure [%]                      100.0
Total Fees Paid                      10448.427653
Max Drawdown [%]                        44.887094
Max Drawdown Duration                       528.0
Total Trades                                   26
Total Closed Trades                            26
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                 50.0
Best Trade [%]                          51.620361
Worst Trade [%]                        -12.704292
Avg Winning Trade [%]                   21.875969
Avg Losing Trade [%]                    -4.960688
